# Phase-End Project: Creating a Shopping App Using Python

Problem Scenario: You have to develop a shopping application or e-commerce application which has login and public login features on the Python platform. The applications that have been developed should also include categories, such as 3–4 for footwear, clothing, electronics, etc. It should be possible to add and update categories in the application. Additionally, it must contain a feature that allows you to add or remove items from your cart. Finally, the program needs to support a variety of payment options, including UPI and debit cards. This should be only backend implementation, and UX/UI and database connectivity are not required.

### Guidelines:

	1.	A welcome message should initially be displayed in the e-commerce application, such as "Welcome to the Demo Marketplace".
	2•	User login and admin login should be created once the code for the welcome message has been written. For the creation of demo login and admin login, demo databases for those should be created for the user and admin verification, and session id creation.
	3•	After databases are created, create admin and user logins. It is necessary to construct a sample product catalog with three to four product categories, such as Boots, Coats, Jackets, and Caps. The product id, name, category id, and price should all be present for each item in the dummy database of the catalog. Both users and administrators can view the catalog.
	
    

## Approach taken
- Create a Dummy DB class with attributes: Users, Categories, Products, and catalogs are dict for quicker lookup.
- Catalogs hold what's been displayed for sale
- Dummy data are JSON files in `/data/*.json` 
- Load functions are in the file `utils.json`

In [1]:
import utils

# Create DB & demo data.
from models.database import DummyEcommerceDB

utils.print_header("Welcome to the Demo Marketplace")

# Instance of dummyDB 
db = DummyEcommerceDB()

# Load demo data: Users
utils.load_users(db.users)
db.users


Welcome to the Demo Marketplace
Loading demo users from JSON file ... 

Total demo users: 3 



{'carmen@test.com': {'id': 1,
  'name': 'Carmen',
  'email': 'carmen@test.com',
  'password': 'abcde1',
  'admin': 1},
 'maria@test.com': {'id': 2,
  'name': 'Maria',
  'email': 'maria@test.com',
  'password': 'abcde2',
  'admin': 0},
 'jonh@test.com': {'id': 3,
  'name': 'Jonh',
  'email': 'jonh@test.com',
  'password': 'abcde2',
  'admin': 1}}

### Approach to register and authenticate a user 

At first, I added the function login() to the class DummyEcommerceDB but then moved it to the file utils.

It feels that authentications shouldn't be tied to a class, probably it's okay to keep functions in the DB dummy for adding (registering) a user or adding/updating/removing category, product or user. 

In [2]:
# To register an user , 1= Admin, 0 = No admin 
db.register( "carmencita", "carmencita@test.com", "abcde2", 1)
print(db.users["carmencita@test.com"])

# To login use an existing one, and pass as an argument db.users 
# utils.login("carmen@test.com", "abcde1", db.users) 

{'id': 4, 'name': 'carmencita', 'email': 'carmencita@test.com', 'password': 'abcde2', 'admin': 1}


### Using input() to either login or register a user.
- To authenticate utils.login() (or db.login()) 
- To register utils.register()

In [6]:
import utils
from models.database import User

utils.print_header("(L)OGIN | (R)EGISTER")
choice = input("Press L or R to continue... ")
choice = choice.upper()

print("\n")

if choice == "L":
    utils.print_header("LOGIN SCREEN")
    email = input("Email: ")
    password = input("Password: ")
    
    user_data = utils.login(email, password, db.users)[2] #for debugging/testing purposes I return a tuple, user data is position 2
    # print(user_data)
    if  user_data:
        current_user = User(**user_data)
        print(f"\nLogin successfully | Welcome back {current_user.name}")
    else:
        print("Login failed. Invalid Email or Password. Please try again.")

else:
    utils.print_header("SIGNUP SCREEN")
    
    name = input("Name: ")
    email = input("Email: ")
    password = input("Password: ")

    user_data = db.register(name,email, password)[2]
    if user_data:
        current_user = User(**user_data)

    print(f"\nRegistered successfully | Welcome {current_user.name}")
    

(L)OGIN | (R)EGISTER


### Loading Catalog, Categories and Products 

In [31]:
# Import.
from models.database import Category 
from models.database import Product

import utils

## Uncomment these lines first to run just this row. We need db, a logged user, and catalogs to continue
# from models.database import DummyEcommerceDB
# db = DummyEcommerceDB()
# print(isinstance(db, DummyEcommerceDB))

# Load demo data 
# utils.load_users(db.users)

# Load demo data: Catalogs, Categories, and Products  
utils.print_header("Product's Catalog")
if len(db.catalogs) == 0:
    catalog = utils.load_catalog(db)['catalog']
else:
    catalog = db.catalogs
    
# print([*catalog.keys()])
print(f'Categories: \n {[*db.categories.keys()]}')

# Print products
print(f"Products: \n {[product['name'] for product in db.products.values()]}")
# same as print(list(catalog.keys))

# To login use
#"carmen@test.com", "abcde1"
# 0: client, 1: admin

# Or to set a current_user 
# current_user = utils.current_user("carmen@test.com", "abcde1", db.users)
# print(f'current user {current_user.email}')

Product's Catalog
Categories: 
 ['books', 'clothing']
Products: 
 ['Python Crash Course: A Hands-On, Project-Based Introduction to Programming', 'Fluent Python: Clear, Concise, and Effective Programming', 'Introduction to Machine Learning With Python', 'AUTOMET Womens Faux Leather Jackets']


###  Display product Catalog
 - Using ipywidgets as interative HTML widgets, so consumer can click (+) to add or (-) to remove an item to/from a cart. 
 - Generate dynamic variables to trigger onClick events for each product_id. I decided to use Button instead of Checkboxes. Note: I left the Checkbox example so you can see my thought process.  

In [16]:
# Cart
import utils
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import HBox

# # Uncomment these lines first to run just this row
## ====================
# # Create DB & demo data.
from models.database import DummyEcommerceDB
from models.database import Category 
from models.database import Product
db = DummyEcommerceDB()
# print(isinstance(db, DummyEcommerceDB))

# # Option 1: Login an user, it returns an instance of class User()
# utils.load_users(db.users)
# current_user = utils.current_user("carmen@test.com", "abcde1", db.users)
# print(f'current user {current_user.email}') # return tuple

# # Option 2: Login an user, it returns dict with user_data
utils.load_users(db.users)
user_data = utils.login("carmen@test.com", "abcde1", db.users)[2]
print(f'current user {user_data["email"]}')
email = user_data["email"]

# # ===================

# Get current's user email as global variable
# email = current_user.email[0] 

utils.print_header("Product's Catalog")
if len(db.catalogs) == 0:
    catalog = utils.load_catalog(db)['catalog']
else:
    catalog = db.catalogs

# Define widgets
order_title = widgets.Output()
out = widgets.Output()
order_display = widgets.VBox([order_title, out])
button = widgets.Button(description="View Cart")

# Define button functions 
def on_view_cart_clicked(b):
    data = utils.cart_data(cart, email)
    display(data['output'])


@out.capture()
def on_add_clicked(b):
    # print(f'clicked on add: {b.description}')
    product_id = int(b.description)
    item = db.products[product_id]

    if email not in cart:
        cart[email] = {}

    if product_id not in cart[email]:
        cart[email][product_id] = {"quantity": 1, "name": item["name"], "price": item["price"]}
    else:
        cart[email][product_id]["quantity"] += 1
    
    item = cart[email][product_id]
        
    print(f'{item["name"]} ${item["price"]} added to your cart')


@out.capture()
def on_remove_clicked(b):
    product_id = int(b.description)

    if email not in cart:
        return
    
    if product_id in cart[email]:
        item = cart[email][product_id]

        if item and item["quantity"] > 0:
            item["quantity"] -= 1
        else:
            print("Item does not exists in cart")
            return
        print(f'{item["name"]} ${item["price"]} removed from your cart')
    else:
        print("Item is not in your cart")

btn_products = []
cart = {}
for key, category in catalog.items():
    # print(category)
    products = category.get("products", None)
    category = Category(category["id"], category["name"], category["description"])
    utils.print_header(f"Category: {category.id} | {category.name} | {category.description}")

    # Prefix to generate dynamic variables
    prefix = "btn_" + str(category.id)

    for product_data in products:
        # In theory it isn't neccessary , just to practice more python
        product = Product(**product_data)
        
        # Dynamic variable for + button
        btn_product = prefix + str(product.id)

        if product.id not in db.products:
            db.products[product.id] = {**product_data}
        
        print(f"ID: {product.id} -- \n{product.title}")
        print(f"by {product.author}")
        print(f"$ {product.price}\n {'-'*40}")

        # Build add and del buttons
        btn_add = btn_product + "_add"
        btn_del = btn_product + "_del"
        globals()[btn_add] =  widgets.Button(description=f"{product.id}", tooltip=f"Add {product.id} - {product.title}",value=False, icon='plus')
        globals()[btn_del] =  widgets.Button(description=f"{product.id}", tooltip=f"Remove {product.id} - {product.title}",value=False, icon='remove')
        # btn_products.append({"btn": globals()[btn_add], "product": product})

        globals()[btn_add].on_click(on_add_clicked, False)
        globals()[btn_del].on_click(on_remove_clicked, False)

        ui = widgets.VBox([widgets.HBox([globals()[btn_add], globals()[btn_del]])])
        display(ui)

    print("\n")

with order_title:
    order_title.clear_output()
    utils.print_header("Order Summary")

display(order_display)

# from IPython.display import display
# button = widgets.Button(description="View Cart")
display(button)

# def on_view_cart_clicked(b):
#     data = utils.cart_data(cart, email)
#     display(data['output'])

button.on_click(on_view_cart_clicked)
# display(out)

Loading demo users from JSON file ... 

Total demo users: 3 

current user carmen@test.com
Product's Catalog
Loading DEMO catalog from JSON file :p ... 

Category: 1 | Books | Only technical books
ID: 11 -- 
Python Crash Course: A Hands-On, Project-Based Introduction to Programming
by Eric Matthes
$ 44.99
 ----------------------------------------


ID: 12 -- 
Fluent Python: Clear, Concise, and Effective Programming
by Luciano Ramalho
$ 19.99
 ----------------------------------------


ID: 13 -- 
Introduction to Machine Learning With Python
by Andreas C. Müller
$ 56.99
 ----------------------------------------




Category: 2 | Clothing | Including Jewelry, Jackets, and shoes
ID: 21 -- 
AUTOMET Womens Faux Leather Jackets
by AUTOMET
$ 29.99
 ----------------------------------------


Button(description='View Cart', style=ButtonStyle())

### Display Cart 

Click "View Cart" 

In [17]:
import utils
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="View Cart")
display(button)

def on_view_cart_clicked(b):
    data = utils.cart_data(cart, email)
    display(data['output'])

button.on_click(on_view_cart_clicked)


Button(description='View Cart', style=ButtonStyle())

In [24]:
def checkout(cart, email):
    if email not in cart or not cart[email]:
        print("Cart is empty.")

    return sum( item['quantity'] * item['price'] for product_id, item in cart[email].items())
    # return f"Checkout successful. Total amount: {total_amount}"
    
# cart[email] = {}  # Reset cart
total = checkout(cart, email)
print(f"Total ${total}")


Total $353.91


In [30]:
import ipywidgets as widgets
from ipywidgets import HBox
import pandas as pd

from IPython.display import clear_output, display

out = widgets.Output(layout={'border': '1px solid black'})

from ipywidgets import interact, Dropdown
# payment_options = Dropdown(
#     options=[('PayPal', 1), ('AMEX', 2), ('Venmo', 3)],
#     description='Payment Options:',
# )

# with out:

#     out.clear_output()
#     total = checkout(cart, email)
#     payment_options.observe(value)

# # out

payment_options = [('PayPal', 1), ('AMEX', 2), ('Venmo', 3)]
# payments_dropdown = widgets.Dropdown(
#     options=payment_options,
#     value=payment_options[1],
#     description='Payment Options:',
#     disabled=False,
# )

# display(payments_dropdown)


drop_down = widgets.Dropdown(options=payment_options,
                                description='Choose',
                                disabled=False)

def dropdown_handler(change):
        print(change.new)
        bor = change.new  # This line isn't working
drop_down.observe(dropdown_handler, names='value')
display(drop_down)



Dropdown(description='Choose', options=(('PayPal', 1), ('AMEX', 2), ('Venmo', 3)), value=1)

2


In [ ]:
# def payment_options(change):
#     print(change.new)

# drop_down.observe()    